### Python Code of the Udacity free course Intro to data analysis

### Lesson 1: Data Analysis Process

In this first part, the code involves process of data wrangling (data acquisition and cleaning), exploration (building intuition and finding patterns), drawing conclusion (or make predictions) and comunicating the results. The soil dataset used is one provided by Udacity, which has information on students' engagements, enrollments and project submissions.

#### Step 1: Load libraries and dataset

In [3]:
## Install libraries
#!pip install unicodecsv

In [2]:
## Load libraries
import unicodecsv
from datetime import datetime as dt
from collections import defaultdict
import numpy as np

In [4]:
# Define function to read csv
def read_csv(filename):
    with open(filename, 'rb') as f:
        reader = unicodecsv.DictReader(f)
        return list(reader)

In [5]:
# Load data
enrollments = read_csv('C:/Users/neliq/Desktop/Intro-to-data-analysis/enrollments.csv')
engagements = read_csv('C:/Users/neliq/Desktop/Intro-to-data-analysis/daily_engagement.csv')
submissions = read_csv('C:/Users/neliq/Desktop/Intro-to-data-analysis/project_submissions.csv')

#### Step 2: Fixing column names and data types


In [6]:
## Rename the column acct
for eng in engagements:
    eng['account_key'] = eng['acct']
    del[eng['acct']]

In [7]:
enrollments[1]

{'account_key': '448',
 'status': 'canceled',
 'join_date': '2014-11-10',
 'cancel_date': '2015-01-14',
 'days_to_cancel': '65',
 'is_udacity': 'True',
 'is_canceled': 'True'}

In [8]:
## Define functions

## 1. Change date string, and returns a Python datetime object
## If there's no date given, returns None

def parse_date(date):
    if date == '':
        return None
    else:
        return dt.strptime(date, '%Y-%m-%d')
    
## 2. Change a string or empty string which represents an integer,
## and returns an in or None

def parse_maybe_int(i):
    if i == '':
        return None
    else:
        return int(i)
    
## Apply the functions and change data types of the enrollments dataset
for enrollment in enrollments:
    enrollment['cancel_date'] = parse_date(enrollment['cancel_date'])
    enrollment['days_to_cancel'] = parse_maybe_int(enrollment['days_to_cancel'])
    enrollment['is_canceled'] = enrollment['is_canceled'] == 'True'
    enrollment['is_udacity'] = enrollment['is_udacity'] == 'True'
    enrollment['join_date'] = parse_date(enrollment['join_date'])

## Clean up the data types in the engagement table
for engagement_record in engagements:
    engagement_record['lessons_completed'] = int(float(engagement_record['lessons_completed']))
    engagement_record['num_courses_visited'] = int(float(engagement_record['num_courses_visited']))
    engagement_record['projects_completed'] = int(float(engagement_record['projects_completed']))
    engagement_record['total_minutes_visited'] = float(engagement_record['total_minutes_visited'])
    engagement_record['utc_date'] = parse_date(engagement_record['utc_date'])
    
## Clean up the data types in the submissions table
for submission in submissions:
    submission['completion_date'] = parse_date(submission['completion_date'])
    submission['creation_date'] = parse_date(submission['creation_date'])


## Check 
enrollments[0]
engagements[0]
submissions[0]

{'creation_date': datetime.datetime(2015, 1, 14, 0, 0),
 'completion_date': datetime.datetime(2015, 1, 16, 0, 0),
 'assigned_rating': 'UNGRADED',
 'account_key': '256',
 'lesson_key': '3176718735',
 'processing_state': 'EVALUATED'}

#### Step 3: Investigating the data

Find the total number of enrollments, engagements and project submissions as well as the unique values for each. 

In [9]:
def get_unique_students(data):
    unique_students = set()
    for data_point in data:
        unique_students.add(data_point['account_key'])
    return unique_students


print('Enrollments:', len(enrollments))
unique_enrolled_students = get_unique_students(enrollments)
print('Unique enrollments:', len(unique_enrolled_students))
print('Engagements:', len(engagements))
unique_engagement_students = get_unique_students(engagements)
print('Unique engagements:', len(unique_engagement_students))
print('Submissions:', len(submissions))
unique_project_submitters = get_unique_students(submissions)
print('Unique submissions:', len(unique_project_submitters))

Enrollments: 1640
Unique enrollments: 1302
Engagements: 136240
Unique engagements: 1237
Submissions: 3642
Unique submissions: 743


#### Missing engagement records

In this case, there are few enrollment missing in the unique_engagement_students records. We have 1302 enrollemnts and 1237 engagements. They should be the same. Below we are printing one example and it is possible to observe that those are account that were cancelled in the same day. 

In [10]:
for enrollment in enrollments:
    student = enrollment['account_key']
    if student not in unique_engagement_students:
        print(enrollment)
        break

{'account_key': '1219', 'status': 'canceled', 'join_date': datetime.datetime(2014, 11, 12, 0, 0), 'cancel_date': datetime.datetime(2014, 11, 12, 0, 0), 'days_to_cancel': 0, 'is_udacity': False, 'is_canceled': True}


#### Check for more problem records

Here we are trying to find more records that are not in the engagement records and cancelled the course in the same day. The print of the records revealed another problem, which was that there are a few udacity test account, which should be removed

In [11]:
num_problem_students = 0
for enrollment in enrollments:
    student = enrollment['account_key']
    if (student not in unique_engagement_students and enrollment['join_date'] != enrollment['cancel_date']):
        print(enrollment)
        num_problem_students += 1

num_problem_students

{'account_key': '1304', 'status': 'canceled', 'join_date': datetime.datetime(2015, 1, 10, 0, 0), 'cancel_date': datetime.datetime(2015, 3, 10, 0, 0), 'days_to_cancel': 59, 'is_udacity': True, 'is_canceled': True}
{'account_key': '1304', 'status': 'canceled', 'join_date': datetime.datetime(2015, 3, 10, 0, 0), 'cancel_date': datetime.datetime(2015, 6, 17, 0, 0), 'days_to_cancel': 99, 'is_udacity': True, 'is_canceled': True}
{'account_key': '1101', 'status': 'current', 'join_date': datetime.datetime(2015, 2, 25, 0, 0), 'cancel_date': None, 'days_to_cancel': None, 'is_udacity': True, 'is_canceled': False}


3

#### Check number of udacity test accounts

Six records were found. These need to be removed.

In [12]:
udacity_test_accounts = set()
for enrollment in enrollments:
    if enrollment['is_udacity']:
        udacity_test_accounts.add(enrollment['account_key'])
len(udacity_test_accounts)

6

#### Remove udacity test accounts and create new clean datasets

In [13]:
def remove_udacity_accounts(data):
    non_udacity_data = []
    for data_point in data:
        if data_point['account_key'] not in udacity_test_accounts:
            non_udacity_data.append(data_point)
    return non_udacity_data

non_udacity_enrollments = remove_udacity_accounts(enrollments)
non_udacity_engagements = remove_udacity_accounts(engagements)
non_udacity_submissions = remove_udacity_accounts(submissions)

print(len(non_udacity_enrollments))
print(len(non_udacity_engagements))
print(len(non_udacity_submissions))

1622
135656
3634


#### Step 4:  Exploration phase

Calculate the number of students who did not cancel before a week, remove free trials cancels and get data from the first week.

In [21]:
paid_students = {}
for enrollment in non_udacity_enrollments:
    if (not enrollment['is_canceled'] or
            enrollment['days_to_cancel'] > 7):
        account_key = enrollment['account_key']
        enrollment_date = enrollment['join_date']
        if (account_key not in paid_students or
                enrollment_date > paid_students[account_key]):
            paid_students[account_key] = enrollment_date
len(paid_students)

995

In [66]:
max(total_minutes_by_account.items(), key=lambda pair: pair[1])

('163', 3564.7332644989997)

In [67]:
student_with_max_minutes = None
max_minutes = 0

for student, total_minutes in total_minutes_by_account.items():
    if total_minutes > max_minutes:
        max_minutes = total_minutes
        student_with_max_minutes = student

max_minutes

for engagement_record in paid_engagement_in_first_week:
    if engagement_record['account_key'] == student_with_max_minutes:
        print(engagement_record)

{'utc_date': datetime.datetime(2015, 7, 9, 0, 0), 'num_courses_visited': 4, 'total_minutes_visited': 850.519339666, 'lessons_completed': 4, 'projects_completed': 0, 'account_key': '163'}
{'utc_date': datetime.datetime(2015, 7, 10, 0, 0), 'num_courses_visited': 6, 'total_minutes_visited': 872.633923334, 'lessons_completed': 6, 'projects_completed': 0, 'account_key': '163'}
{'utc_date': datetime.datetime(2015, 7, 11, 0, 0), 'num_courses_visited': 2, 'total_minutes_visited': 777.018903666, 'lessons_completed': 6, 'projects_completed': 0, 'account_key': '163'}
{'utc_date': datetime.datetime(2015, 7, 12, 0, 0), 'num_courses_visited': 1, 'total_minutes_visited': 294.568774, 'lessons_completed': 2, 'projects_completed': 0, 'account_key': '163'}
{'utc_date': datetime.datetime(2015, 7, 13, 0, 0), 'num_courses_visited': 3, 'total_minutes_visited': 471.2139785, 'lessons_completed': 1, 'projects_completed': 0, 'account_key': '163'}
{'utc_date': datetime.datetime(2015, 7, 14, 0, 0), 'num_courses_vi

In [68]:
## Get data from the first week
def within_one_week(join_date, engagement_date):
    time_delta = engagement_date - join_date
    return time_delta.days >= 0 and time_delta.days < 7

paid_engagement_in_first_week = []

for engagement_record in paid_engagement:
    account_key = engagement_record['account_key']
    join_date = paid_students[account_key]
    engagement_record_date = engagement_record['utc_date']

    if within_one_week(join_date, engagement_record_date):
         paid_engagement_in_first_week.append(engagement_record)

len(paid_engagement_in_first_week)

6919

#### Step 5: Exploring students engagements

In [61]:
engagement_by_account = defaultdict(list)
for engagement_record in paid_engagement_in_first_week:
    account_key = engagement_record['account_key']
    engagement_by_account[account_key].append(engagement_record)

In [62]:
total_minutes_by_account = {}

for account_key, engagement_for_student in engagement_by_account.items():
        total_minutes = 0
        for engagement_record in engagement_for_student:
            total_minutes += engagement_record['total_minutes_visited']
        total_minutes_by_account[account_key] = total_minutes
        
total_minutes_by_account

{'0': 494.88049616599994,
 '1': 18.576384666670002,
 '2': 0.0,
 '3': 0.0,
 '4': 33.3214046667,
 '5': 329.7578566663,
 '6': 780.4545511666701,
 '7': 104.20388850009999,
 '8': 989.113641833,
 '9': 448.471384167,
 '10': 130.12347833367,
 '11': 0.0,
 '12': 179.4719088333,
 '13': 1013.3833969996999,
 '14': 65.6221875,
 '15': 983.375040335,
 '16': 119.12030049999998,
 '17': 235.49969150033,
 '18': 155.1361575,
 '19': 0.0,
 '20': 447.93897783336996,
 '21': 931.1036911666699,
 '22': 657.2052335000001,
 '23': 1591.3228143334,
 '24': 943.188117167,
 '25': 0.0,
 '26': 430.801675833,
 '27': 1579.12122666663,
 '28': 766.256315667,
 '29': 556.1906033333,
 '30': 69.6578351667,
 '31': 0.0,
 '32': 123.2915048333,
 '33': 253.9870258334,
 '34': 180.413814,
 '35': 765.6402170004,
 '36': 809.2138958339,
 '37': 1378.195091668,
 '38': 0.0,
 '39': 1001.5888595,
 '40': 478.21952616690004,
 '41': 511.925391,
 '42': 576.4643026663,
 '43': 88.822038,
 '44': 0.0,
 '45': 0.0,
 '46': 26.1189351667,
 '47': 854.828888

In [63]:
mean_students = sum(total_minutes_by_account.values())/len(total_minutes_by_account.values())
min_students = min(total_minutes_by_account.values())
max_students = max(total_minutes_by_account.values())
print("Mean: ", mean_students) 
print('Min:', min_students)
print('Max: ', max_students)

Mean:  306.7083267534284
Min: 0.0
Max:  3564.7332644989997


In [72]:
def group_data(data, key_name):
    grouped_data = defaultdict(list)
    for data_point in data:
        key = data_point[key_name]
        grouped_data[key].append(data_point)
    return grouped_data

engagement_by_account = group_data(paid_engagement_in_first_week,
                                   'account_key')

def sum_grouped_items(grouped_data, field_name):
    summed_data = {}
    for key, data_points in grouped_data.items():
        total = 0
        for data_point in data_points:
            total += data_point[field_name]
        summed_data[key] = total
    return summed_data

total_minutes_by_account = sum_grouped_items(engagement_by_account,
                                             'total_minutes_visited')

import numpy as np

def describe_data(data):
    print('Mean:', np.mean(data))
    print('Standard deviation:', np.std(data))
    print('Minimum:', np.min(data))
    print('Maximum:', np.max(data))

describe_data(total_minutes_by_account.values())

TypeError: unsupported operand type(s) for /: 'dict_values' and 'int'